### Packages import


In [ ]:
# import os
# import glob
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import time
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

from torchvision import models, transforms
from sklearn.model_selection import train_test_split

from skimage import io
from PIL import Image

In [ ]:
# importing all the required libraries
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import skimage.io as io
from skimage.transform import rotate, AffineTransform, warp
from skimage.util import random_noise
from skimage.filters import gaussian
import matplotlib.pyplot as plt
% matplotlib inline

## Data Loading and Preprocessing

In [ ]:
# !unzip /content/drive/MyDrive/FDL/train_images.zip -d /content/drive/MyDrive/FDL/augmented_dataset/
# !unzip /content/drive/MyDrive/FDL/train_masks_zipped.zip -d /content/drive/MyDrive/FDL/augmented_dataset/

In [ ]:
directory = ""
train_images = "train_images/*.png" # initial images start by 0 # transformed will start by "transformation_name"
train_masks = "train_masks/*.png"

In [ ]:
# Load image and mask file names, and make sure they are ordered the same
image_list = list(glob.glob(os.path.join(directory + train_images)))
mask_list = list(glob.glob(os.path.join(directory + train_masks)))
image_list = np.sort(image_list)
mask_list = np.sort(mask_list)

In [ ]:
assert len(mask_list) == len(image_list), print(len(mask_list), len(image_list))

In [ ]:
from skimage.io import imsave, imread


image_directory = "augmented_dataset/train_images/"
mask_directory = "augmented_dataset/train_masks/"
# *255).astype(np.uint8)


def rotate(img, angle= 45, mode= "wrap"):
  img_roated = rotate(img, angle=angle, mode = mode)
  # io.imshow(img_roated)
  return img_rotated

def shift(img, translation = (25,25), mode= "wrap"):
    transform = AffineTransform(translation=translation)
    wrapShift = warp(img,transform,mode=mode)
    # plt.imshow(wrapShift)
    # plt.title('Wrap Shift')
    return wrapShift

def flip(img, mode):
    if mode == "left_right":
      flipLR = np.fliplr(img) # left right
      # plt.imshow(flipLR)
    else:
      flipLR = np.flipud(img) # up down
      # plt.imshow(flipLR)
    return flipLR

def add_noise(img, sigma=0.155):
    noisyRandom = random_noise(img,var=sigma**2)
    # plt.imshow(noisyRandom)
    return noisyRandom

def blur_image(img, sigma=1, multichannel=True):
    blurred = gaussian(img,sigma=sigma,multichannel=multichannel)
    # plt.imshow(blurred)
    return blurred

def save_img(img, dir, img_name, prefix="XXXX"):
    path = str(dir + prefix + img_name)
    # img = (img * 255).astype(np.uint8)
    print(path)
    io.imsave(path, img)

In [ ]:
import random



for i, (image, mask) in enumerate(zip(image_list, mask_list)):
    img = io.imread(image)
    msk = io.imread(mask)

    image_name = image.split("/")[-1:][0]
    mask_name = mask.split("/")[-1:][0]

    print(image)
    print(mask)

    augmentation = random.choice(["flip", "blur", "shift", "noise"])


    #shift
    if augmentation == "shift":
        img_shifted = shift(img, translation = (25,25))
        msk_shifted = shift(msk, translation = (25,25))
        save_img(img_shifted, image_directory, image_name, prefix="shifted_")
        save_img(msk_shifted, mask_directory, mask_name, prefix="shifted_")   

    # # flip
    elif augmentation == "flip":
        if i % 2 == 0:
            img_flipped = flip(img, "left_right")
            mask_flipped = flip(msk, "left_right")
            save_img(img_flipped, image_directory, image_name, prefix="lr_")
            save_img(mask_flipped, mask_directory, mask_name, prefix="lr_")

        else:
            img_flipped = flip(img, "up_down")
            mask_flipped = flip(msk, "up_down")
            save_img(img_flipped, image_directory, image_name, prefix="ud_")
            save_img(mask_flipped, mask_directory, mask_name, prefix="ud_")
    
    
    #### ----- only modify the image ----- #### (but still save mask for consistency)

    # add noise
    elif augmentation == "noise":
        img_noisy = add_noise(img)
        save_img(img_noisy, image_directory, image_name, prefix="noisy_")
        save_img(msk, mask_directory, mask_name, prefix="noisy_") # mask as such

    # blur
    elif augmentation == "blur":
        img_blurred = blur_image(img)
        save_img(img_blurred, image_directory, image_name, prefix="blurred_")
        save_img(msk, mask_directory, mask_name, prefix="blurred_") # mask as such

/content/drive/MyDrive/FDL/augmented_dataset/train_images/00001.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/00001.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/shifted_00001.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/shifted_00001.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/00002.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/00002.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/blurred_00002.png


/content/drive/MyDrive/FDL/augmented_dataset/train_masks/blurred_00002.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/00003.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/00003.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/lr_00003.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/lr_00003.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/00004.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/00004.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/noisy_00004.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/noisy_00004.png


/content/drive/MyDrive/FDL/augmented_dataset/train_images/00005.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/00005.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/noisy_00005.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/noisy_00005.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/00006.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/00006.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/noisy_00006.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/noisy_00006.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/00007.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/00007.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/blurred_00007.png


/content/drive/MyDrive/FDL/augmented_dataset/train_masks/blurred_00007.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/00008.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/00008.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/blurred_00008.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/blurred_00008.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/00009.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/00009.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/shifted_00009.png


/content/drive/MyDrive/FDL/augmented_dataset/train_masks/shifted_00009.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/00010.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/00010.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/ud_00010.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/ud_00010.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/00011.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/00011.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/blurred_00011.png


...

/content/drive/MyDrive/FDL/augmented_dataset/train_masks/lr_02645.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/02646.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/02646.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/blurred_02646.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/blurred_02646.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/02647.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/02647.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/noisy_02647.png


/content/drive/MyDrive/FDL/augmented_dataset/train_masks/noisy_02647.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/02648.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/02648.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/ud_02648.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/ud_02648.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/02649.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/02649.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/noisy_02649.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/noisy_02649.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/02650.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/02650.png


/content/drive/MyDrive/FDL/augmented_dataset/train_images/blurred_02650.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/blurred_02650.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/02651.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/02651.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/blurred_02651.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/blurred_02651.png


/content/drive/MyDrive/FDL/augmented_dataset/train_images/02652.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/02652.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/blurred_02652.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/blurred_02652.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/02653.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/02653.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/noisy_02653.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/noisy_02653.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/02654.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/02654.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/ud_02654.png


/content/drive/MyDrive/FDL/augmented_dataset/train_masks/ud_02654.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/02655.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/02655.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/noisy_02655.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/noisy_02655.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/02656.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/02656.png
/content/drive/MyDrive/FDL/augmented_dataset/train_images/noisy_02656.png
/content/drive/MyDrive/FDL/augmented_dataset/train_masks/noisy_02656.png
